In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

### To install kerastuner, the tensorflow version must be greater than 2.0. To find out the version of tensor flow, in the anaconda prompt type activate environment, enter print(tensorflow.__version__) and then enter exit(). Also Kerastuner helps to perform hyperparameter tuning which will help us to no. of hidden layers and neurons. the RandomSearch helps to find out the optimum no. of neurons and the no. of hidden layers in the model.

In [2]:
df=pd.read_csv('Real_combine.csv')

In [3]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [9]:
X=df.iloc[:,:-1]  ##Independent/ input features
y=df.iloc[:,-1] ##Dependent / output features

In [5]:

def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model                                       

# Explanation:
1. hp is an alias for hypermodel which is a class in Kerastuner that lets us define the model with a searchable space and build it.hp is sent as a parameter in the user defined function build_model.
2. hp.Int, hp.float and hp.choice are used to define a search space that accepts an integer, float and category respectively.
3. https://keras.io/api/keras_tuner/hyperparameters/ - hyperparameter documentation. https://keras.io/keras_tuner/ - keras tuner documentation.
4. hp.Int in the for loop creates iterates through 2 to 20, creating the neural layers btwn 2 to 20 and since the default step size value is not defined, it will be the minimum value itself. So the hp.Int will be 2,4 ,6..20. hence the value in the for loop iterates through this even values.
5. Once the no. of neural layers is defined, the next step is to add neurons to the layers, we choose different different values using hp.Int which will choose values like 32,64..512 etc. with activation function as relu which works the best in hidden and input layers avoid exploding gradient problem.
6. model.add(layers.Dense(1,activation='linear')) is an output layer and the activation function is linear which is usually used for regression problems.
7. Compiling using Adam optimizer and choosing the learning rate, hp.choice will choose one of the learning rates. Loss= mean absolute error is given because it's a regression problem.

## Hyperparameters:
1. How many no. of hidden layers we should have.
2. How many no. of neurons we should have.
3. Learning rate.

In [6]:
tuner=RandomSearch(
   build_model,
   objective='val_mean_absolute_error',
   max_trials=5,
   executions_per_trial=3,
   directory='project',
   project_name='Air Quality Index')

 In the above code we are creating the instance of the class RandomSearch and calling the instance of the build model where in we have passed the alias of the hyperparameter. Here the objective is val_mean_absolute_error and for a classification problem it will be accuracy. Max trials is and for each trial we run this 3 times with all the parameters so in total there will be 15 iterations. Also the directory is initialised so that all the weights are stored in this as the training / search of the model is happening.

In [7]:
tuner.search_space_summary()

In [10]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [11]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - ETA: 0s - loss: 97.9305 - mean_absolute_error: 97.930 - ETA: 0s - loss: 92.7974 - mean_absolute_error: 92.797 - 1s 27ms/step - loss: nan - mean_absolute_error: nan - val_loss: 64.4953 - val_mean_absolute_error: 64.4953
Epoch 2/5
24/24 [==============================] - ETA: 0s - loss: 67.1051 - mean_absolute_error: 67.105 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 5ms/step - loss: nan - mean_absolute_error: nan - val_loss: 66.0326 - va

C:\Users\Deeksha Kotian\Anaconda3\envs\deeplearning\lib\site-packages\kerastuner\engine\metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - ETA: 0s - loss: 107.2995 - mean_absolute_error: 107.299 - ETA: 0s - loss: 95.9269 - mean_absolute_error: 95.9269  - 0s 17ms/step - loss: nan - mean_absolute_error: nan - val_loss: 63.8652 - val_mean_absolute_error: 63.8652
Epoch 2/5
24/24 [==============================] - ETA: 0s - loss: 74.0136 - mean_absolute_error: 74.013 - ETA: 0s - loss: 69.4893 - mean_absolute_error: 69.489 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 9ms/step - l

C:\Users\Deeksha Kotian\Anaconda3\envs\deeplearning\lib\site-packages\kerastuner\engine\metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - ETA: 0s - loss: 108.1684 - mean_absolute_error: 108.168 - ETA: 0s - loss: nan - mean_absolute_error: nan          - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 18ms/step - loss: nan - mean_absolute_error: nan - val_loss: 86.4784 - val_mean_absolute_error: 86.4784
Epoch 2/5
24/24 [==============================] - ETA: 0s - loss: 92.1493 - mean_absolute_error: 92.149 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - loss: nan - m

...

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - ETA: 0s - loss: 154.7855 - mean_absolute_error: 154.785 - ETA: 0s - loss: 110.4055 - mean_absolute_error: 110.405 - ETA: 0s - loss: 109.8781 - mean_absolute_error: 109.878 - ETA: 0s - loss: nan - mean_absolute_error: nan          - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - 1s 29ms/step - loss: nan - mean_absolute_error: nan - val_loss: 67.2208 - val_mean_absolute_error: 67.2208
Epoch 2/5
24/24 [=====

C:\Users\Deeksha Kotian\Anaconda3\envs\deeplearning\lib\site-packages\kerastuner\engine\metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - ETA: 0s - loss: 146.8245 - mean_absolute_error: 146.824 - ETA: 0s - loss: 107.3087 - mean_absolute_error: 107.308 - ETA: 0s - loss: nan - mean_absolute_error: nan          - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - 1s 40ms/step - loss: nan - mean_absolute_error: nan - val_loss: 65.6163 - val_mean_absolute_error: 65.6163
Epoch 2/5
24/24 [===============

C:\Users\Deeksha Kotian\Anaconda3\envs\deeplearning\lib\site-packages\kerastuner\engine\metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


INFO:tensorflow:Oracle triggered exit


In [13]:
tuner.results_summary()